In [1]:
# Using geopandas to spatially join household xy with various geographies

In [7]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon

In [92]:
hh = pd.read_excel(r'J:\Projects\Surveys\HHTravel\Survey2017\Data\Dataset_2 August 2017\Household\1-Household-v2.xlsx',
                  sheetname='1-Household')

In [31]:
# create a geometry column, required for turning df into a geo df
# df['geometry'] = df.apply(lambda x: Point((float(x.final_lng), float(x.final_lat))), axis=1)
# df.crs = {'init': 'epsg:4326'}    # WGS 84

In [93]:
geog = hh.apply(lambda x: Point((float(x.final_lng), float(x.final_lat))), axis=1)
geog = gpd.GeoSeries(geog)
geog.crs = {'init': 'epsg:4326'}    # WGS 84
hh['lat_lon_geog'] =  geog
hh['geometry'] = geog.to_crs(epsg='2285') # Replace default geometry field with the projected epsg=2285 projection to match shapefiles
hh.crs = {'init': 'epsg:2285'}

In [150]:
# Load geography layers
tract = gpd.GeoDataFrame.from_file(r'W:\geodata\census\Tract\tract2010.shp')
tract.crs = {'init' :'epsg:2285'}

block = gpd.GeoDataFrame.from_file(r'W:\geodata\census\Block\block2010.shp')
block.crs = {'init' :'epsg:2285'}

block_group = gpd.GeoDataFrame.from_file(r'R:\Brice\blockgrp2010.shp')
block_group.crs = {'init' :'epsg:2285'} # PUMA is not a projected shapefile; need to project 

puma = gpd.GeoDataFrame.from_file(r'W:\geodata\census\PUMAs\reg10puma.shp')
puma.crs = {'init' :'epsg:4326'}
puma['geometry'] = puma['geometry'].to_crs(epsg='2285')
puma.crs = {'init': 'epsg:2285'}

rgc = gpd.GeoDataFrame.from_file(r'R:\Brice\urbcen.shp')
rgc.crs = {'init' :'epsg:2285'}

urbv = gpd.GeoDataFrame.from_file(r'W:\geodata\political\urbanvillages.shp')
urbv.crs = {'init' :'epsg:2285'}

taz = gpd.GeoDataFrame.from_file(r'W:\geodata\forecast\taz2010.shp')
taz.crs = {'init' :'epsg:2285'}

In [151]:
def spatial_join(gdf1, gdf2, keep_field, rename_field, crs):
    """Spatial join two geodataframes, left intersect with base on gdf1"""
    df = gpd.sjoin(gdf1, gdf2[['geometry',keep_field]], how="left", op='intersects')
    df = df.rename(columns={keep_field: rename_field})
    df = df.drop(['index_right','index_left'], axis=1)
    df.crs = crs
    
    return df

In [152]:
# Attach tract ID
df = spatial_join(gdf1=hh, gdf2=tract, keep_field='GEOID10', rename_field='final_tract', crs=hh.crs)

# Attach block ID
df = spatial_join(gdf1=df, gdf2=block, keep_field='GEOID10', rename_field='final_block', crs=hh.crs)

# Attach block group ID
df = spatial_join(gdf1=df, gdf2=block_group, keep_field='GEOID10', rename_field='final_bg', crs=hh.crs)

# attach puma
df = spatial_join(gdf1=df, gdf2=puma, keep_field='PUMACE10', rename_field='final_puma10', crs=hh.crs)

# attach rgc
df = spatial_join(gdf1=df, gdf2=rgc, keep_field='NAME', rename_field='final_rgcname', crs=hh.crs)

# attach urban village
df = spatial_join(gdf1=df, gdf2=urbv, keep_field='UV_NAME', rename_field='final_uvname', crs=hh.crs)

# attach taz
df = spatial_join(gdf1=df, gdf2=taz, keep_field='TAZ', rename_field='final_hhtaz', crs=hh.crs)

In [155]:
# Drop geometry fields
df = df.drop(['lat_lon_geog','geometry'], axis=1)

# Export to CSV
df.to_csv(r'J:\Projects\Surveys\HHTravel\Survey2017\Data\Dataset_2 August 2017\Household\1-Household-v3-geocoded.csv',
          index=False)